# [Calculate Option Implied Volatility In Python](https://pythoninoffice.com/calculate-option-implied-volatility-in-python/)

## Black Scholes Option Pricing Formula In Python

In [1]:
import scipy.stats
from numpy import sqrt, log, exp, pi

N = scipy.stats.norm.cdf


def Bs_price(c_p, S, K, r, t, sigma):
    d1 = (log(S/K) + (r+sigma**2/2)*t) / (sigma*sqrt(t))
    d2 = d1 - sigma * sqrt(t)
    if c_p == 'c':
        return N(d1) * S - N(d2) * K * exp(-r*t)
    elif c_p == 'p':
        return N(-d2) * K * exp(-r*t) - N(-d1) * S
    else:
        return "Please specify call or put options."

## Calculating Implied Volatility In Python Brute Force Method

In [2]:
ONE_CENT = 0.01
step = 0.0001

def brute_force(c_p, S, K, r, t, market_price):
        _sigma = 0.5
        for i in range(10000): #max number of calculations is 10000
            bs_price = Bs_price(c_p, S, K, r, t, sigma = _sigma)
            diff = market_price - bs_price
            if diff > ONE_CENT:
                _sigma = _sigma + step
            elif diff < 0 and abs(diff) > ONE_CENT:
                _sigma = _sigma - step
            elif abs(diff) < ONE_CENT:
                return _sigma
        return _sigma

## Calculate Option Implied Volatility In Python – Newton Raphson Method


In this context:

* x is σ (sigma), implied volatility that we are trying to solve
* f(x) is a function that is the theoretical (BS) option price – the actual option price. In other words, we want f(x) = BS_price – market_price = 0
* The derivative of f(x), or f'(σ) is actually known as Vega, or the option price sensitivity to implied volatility. We can calculate Vega easily using the below formula. Note the notation N'() is the standard normal probability density function.

$$Vega=SN^\prime(d_1)\sqrt{(T-t)}$$

In [3]:
N_prime=scipy.stats.norm.pdf

In [4]:
MAX_TRY = 1000
def find_iv_newton(c_p, S, K, r, t, market_price):
    _sigma = 0.5
    for i in range(MAX_TRY):
        _bs_price = Bs_price(c_p,S, K, r, t, sigma=_sigma)
        d1 = (log(S/K) + (r+_sigma**2/2)*t) / (_sigma*sqrt(t))
        diff = market_price - _bs_price
        vega = S*N_prime(d1)*sqrt(t)
        if abs(diff) < ONE_CENT:
            return _sigma
        _sigma += diff/vega
    return _sigma

In [5]:
import time
#这里应该是错得
start = time.time()
sigma = brute_force('c',100,120,0.02,30/365, 50.67)
print(sigma)
print(f'finished in {time.time() - start} seconds')


1.4999999999998899
finished in 2.7135889530181885 seconds


In [6]:
Bs_price('c', 100, 120, 0.02, 30/365, sigma)

10.406113194441492

In [7]:
#牛顿法没问题
iv=find_iv_newton('c',100,120,0.02,30/365, 90.67)

In [8]:
Bs_price('c', 100, 120, 0.02, 30/365, iv)

90.6699757666178